## First import some necessary packages

In [1]:
import logging
import pathlib
import sys

import colorcet as cc
import dotenv
import geopandas as gpd
import holoviews as hv
import hvplot.pandas  # noqa: API import
import numpy as np
import pandas as pd
import panel as pn
import pooch
from bokeh.models import PanTool, WheelZoomTool

# Activate Panel extension to make interactive visualizations
pn.extension()

In [2]:
# # Read questions from cloud storage

# import coastal_dynamics as cd

# questions = cd.read_questions(
#     "az://coastal-dynamics/questions/6_cross_shore_transport_hashed.json",
#     storage_options={"account_name": "coclico"},
# )

!pip show holoviews

Name: holoviews
Version: 1.17.1
Summary: Stop plotting your data - annotate your data and let it visualize itself.
Home-page: https://www.holoviews.org
Author: Jean-Luc Stevens and Philipp Rudiger
Author-email: holoviews@gmail.com
License: BSD
Location: C:\Users\khdeb\mambaforge\envs\coastal\Lib\site-packages
Requires: colorcet, numpy, packaging, pandas, panel, param, pyviz-comms
Required-by: geoviews, hvplot


# (Cross-shore) sediment transport
Welcome to the notebook of week 6! The main topic of this ntoebook is cross-shore sediment transport (chapter 7 of the book, with some extra attention for section 7.5 specifically). We will start with a small introduction, followed by a more detailed look into figure 7.21 from the book.

## Introduction
Remember from chapter 5 that the velocity $u$ close to the bed can be assumed to consists of a wave group-averaged component $\bar{u}$, a short-wave-averaged oscillatory component $u_{lo}$ and a short-wave component $u_{hi}$:
$$
u = \bar{u} + u_{lo} + u_{hi}
$$
We are interested in net sediment transport. We can use the third odd velocity moment as a proxy for this (which is a gross simplification, but a workable one):
$$
\left\langle u \left|u\right|^2\right\rangle = 3 \left\langle \bar{u} \left|u_{hi}\right|^2\right\rangle + \left\langle u_{hi} \left|u_{hi}\right|^2\right\rangle + 3 \left\langle u_{lo} \left|u_{hi}\right|^2\right\rangle + ...
$$
Each of these terms is fully explained in the book (so you should definitly read up on this), but in short, each of these terms refers to the following process:
* $3 \left\langle \bar{u} \left|u_{hi}\right|^2\right\rangle$ : transport related to the mean current
* $\left\langle u_{hi} \left|u_{hi}\right|^2\right\rangle$ : transport related to high-frequency waves
* $3 \left\langle u_{lo} \left|u_{hi}\right|^2\right\rangle$ : transport related to low-frequency waves
Once all contributions are known, an equillibrium profile might be drived mathematically. Bowen (1980) does this analyticall by balancing onshore and offshore transport terms. Conversely, each of the velocity moments can be directly measured in a flume, which is what Roelvink and Stive (1989) did.

For the present exercise, we are interested in figure 7.21 and how it changes for different wave conditions and bathymetry. However, we are limited by the lack of direct observations. Tinker et al (2009) presents a solution. They performed a large amount of measurements and fitted a shape function for both mean and oscillatory flow. Though the paper is very interesting (and we definitly recommend checking it out), you are not required to know it for this exercise. You will hear more about it in the Coastal module as well, should you choose it!

## Define shape functions
Below the shape functions by Tinker et al (2009) are defined. You are not expected to know or remember these, they are just here to help us with the exercise.

In [3]:
def Tinker_mean(h, h_b):
    return (-120 * h_b**2) * (h / h_b)**4.3 * np.exp(-9.4 * h / h_b**0.75)

def Tinker_osci(h, h_b):
    return (2.75 * h_b**0.6) * (h / h_b)**3.5 * np.exp(-4.2 * h / h_b**1.05)

def Tinker_total(h, h_b):
    return Tinker_mean(h, h_b) + Tinker_osci(h, h_b)

Let's try these functions for a simple linear profile, assuming a breaker depth of $h_b = 0.5m$.

In [4]:
def show_transport(h, h_b, plot_where="pop-out"):
    """
    change value of 'plot_where' to:
    'inline' if you would like the plot to show in the notebook
    'pop-out' if you would like the plot to show in a new tab (i.e. seperate window)
    """
    
    hb_slider = pn.widgets.FloatSlider(name="Breaking depth", start=0.1 * np.max(h), end=0.9 * np.max(h), value=0.8)

    @pn.depends(hb_slider.param.value)
    def plot(h_b, h=h):
        print(x[np.argwhere(h==h_b)])
        x_b = x[np.argwhere(h==h_b)][0,0]
        
        bath_plot = hv.Curve((x, -h), label='bathymetry').opts(xlabel='x [m]', ylabel='z [m]') * \
                    hv.HLine(0, label='water level').opts(line_dash='dashed') * \
                    hv.VLine(x_b, label='location of breaking').opts(line_dash='dashed')

        bath_plot = bath_plot.opts(show_legend=True)
        
        mean_transport = Tinker_mean(h, h_b)
        osci_transport = Tinker_osci(h, h_b)
        total_transport = Tinker_total(h, h_b)
        
        transport_curves = hv.Curve((x, mean_transport), label='mean transport') * \
                           hv.Curve((x, osci_transport), label='oscillatory transport') * \
                           hv.Curve((x, total_transport), label='total transport')
        
        transport_plot = transport_curves.opts(xlabel='', ylabel='')
        
        p = (bath_plot.opts(
            height=250, width=800, show_grid=True) + \
         transport_plot.opts(
             height=250, width=800, show_grid=True)).opts(shared_axes=False).cols(1)

        return p

    app = pn.Column(hb_slider, plot)
    
    if plot_where == "inline":
        return app
    elif plot_where == "pop-out":
        app.show()
    else:
        print("please use either inline or pop-out for the plot_where variable")

In [5]:
h_b = 0.5

x = np.linspace(0, 50, 100)

h = np.zeros(x.shape)
h[20:81] = np.linspace(0, 1, len(h[20:81]))
h[81:] = 1

In [9]:
logging.getLogger().setLevel(logging.ERROR)

show_transport(h, h_b, plot_where="inline")

[[34.34343434]]


C:\Users\khdeb\mambaforge\envs\coastal\Lib\site-packages\holoviews\core\data\pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type
C:\Users\khdeb\mambaforge\envs\coastal\Lib\site-packages\holoviews\core\data\pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type
C:\Users\khdeb\mambaforge\envs\coastal\Lib\site-packages\holoviews\core\data\pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To a

[[25.25252525]] 
[[22.22222222]] 
[[31.31313131]] 
[[25.25252525]] 
[[13.13131313]] 
[[22.22222222]]

Column
    [0] FloatSlider(end=0.9, name='Breaking depth', start=0.1, value=0.8)
    [1] ParamFunction(function, _pane=HoloViews, defer_load=False)

# Questions about:
- unit of Q
- interpretation of different scenarios